#2. Cargue el conjunto de datos

In [2]:
data = sqlContext.read.csv("/FileStore/tables/cvrud4vb1478717168201/nba.csv", header=True, inferSchema=True, sep=' ')
display(data)

# 3. Entrene un árbol de decisión

In [4]:
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer, MinMaxScaler
'''
data = sqlContext.read.format("com.databricks.spark.csv")\
  .option("header","true")\
  .option("inferSchema", "true")\
  .option("delimiter", ";")\
  .load('/FileStore/tables/30fur5sb1477536120864/credit_german-8665c.csv')
'''
columns = list(data.dtypes)
features = []
data = data.na.drop()
#data2=data
#data = data.drop(data.Class)

for col, typ in columns:
  if col == "Class":
    continue
  if typ == "string":
    idxCol = col+"Idx"
    feaCol = col+"Feature"
    features.append(feaCol)
    stringIndexer = StringIndexer(inputCol = col, outputCol = idxCol)
    indexer = stringIndexer.fit(data)
    data = indexer.transform(data)
    
    encoder = OneHotEncoder(dropLast = False, inputCol = idxCol, outputCol = feaCol)
    data = encoder.transform(data)
  else:
    assembler = VectorAssembler(inputCols = [col], outputCol = str(col+"Num"))
    data = assembler.transform(data)
    features.append(col)
    
assembler = VectorAssembler(inputCols = features, outputCol = "features")
data = assembler.transform(data)
data = data.select("class", "features")
display(data)

In [5]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

labelIndexer = StringIndexer(inputCol="class", outputCol="label").fit(data)

(train, test) = data.randomSplit([0.7, 0.3])

accuracy_metric = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
recall_metric = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="weightedRecall")
f1_metric = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction")

dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth = 3)
pipeline = Pipeline(stages=[labelIndexer, dt])

model = pipeline.fit(train)
predictions = model.transform(test)

print "Accuracy: ",accuracy_metric.evaluate(predictions)
print "Recall:", recall_metric.evaluate(predictions)
print "F1:", f1_metric.evaluate(predictions)
print model.stages[1].toDebugString

In [6]:
compare_df=predictions.select("label", "prediction")

tp=compare_df.filter(compare_df.label==0).filter(compare_df.prediction == 0).count()
fp=compare_df.filter(compare_df.label==0).filter(compare_df.prediction == 1).count()
fn=compare_df.filter(compare_df.label==1).filter(compare_df.prediction == 0).count()
tn=compare_df.filter(compare_df.label==1).filter(compare_df.prediction == 1).count()

import pandas as pd
print ("Confusion Matrix")
CM=pd.DataFrame([[tp,fp],[fn,tn]], columns=['class 1', 'class 2'], index=['class 1','class 2'])
print (CM)

In [7]:
feature_relevance_idx = model.stages[1].featureImportances.indices
feature_relevance_values =model.stages[1].featureImportances.values
feature_relevance_labels = [{columns[i] : feature_relevance_values[j] } for i, j in zip(feature_relevance_idx, range(len(feature_relevance_values)))]
import pprint
pprint.PrettyPrinter().pprint(feature_relevance_labels)

Claramente de los resultados anteriores se puede apreciar que los atributos más relevantes (al menos para el anterior árblol de decisión generado) en orden descendente son:

* Rebotes ofensivos **(OFF)**
* Porcentaje de tiro de tres puntos **(3p%)**
* Robos por juego **(SPG)**
* Rebotes(totales) por juego **(RPG)**
* Asistencias por partido **(APG)**

Tiene sentido los atributos obtenidos, ya que toma como atributos relevantes la cantidad de robos, rebotes y asistencias ya que se espera que un jugador perimetral participe más en jugadas de tipo defensivo así como en lanzamiento de 3 puntos, mientras que jugadores internos se podría esperar que la cantidad de rebotes ofensivos sea significativamente mayor.

In [9]:
print model.stages[1].toDebugString
test_values=test.limit(3).select('features').toPandas().values

def print_features_from_data(data, list_of_features):
  import numpy as np
  aux=np.array([i[0][j] for i in data for j in list_of_features])
  aux=aux.reshape((3,len(list_of_features)))
  print pd.DataFrame(aux, columns=['feature'+str(i) for i in list_of_features], index=['sample'+str(i+1) for i in range(data.shape[0])])

print_features_from_data(test_values, [6,4,8,9,6,10])


Teniendo en cuenta el anterior árbol y el listado de las características que usa el árbol lo podemos seguir para obtener los respectivos resultados de 3 ejemplos dados. A continuación se muestra las reglas seguidas para llegar a la predicción de cada ejemplo.

1.
  * Feature6 <= 1  (False)
  * Feature10 <= 1.2 (True)
  * Feature4 <=0.309 (True)
  * **Predict 1**
  
2.
  * Feature6 <=1 (True)
  * Feature4 <=0 (False)
  * Feature9 <=0.1 (False)
  * **Predict 0**
  
3.
  * Feature6 <=1 (True)
  * Feature4 <=0 (Flse)
  * Feature9 <=0.1 (False)
  * **Predict 0**

In [11]:
display(model.transform(test.limit(3)).select('label','prediction'))

# 4. Complejidad del modelo

In [13]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
def func(md):
  dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth = md)
  pipeline = Pipeline(stages=[labelIndexer, dt])
  model = pipeline.fit(train)
  
  predictions = model.transform(train)
  accuracy = evaluator.evaluate(predictions)
  t1 = 1.0-accuracy
  
  predictions = model.transform(test)
  accuracy = evaluator.evaluate(predictions)
  t2 = 1.0-accuracy
  return (t1, t2, md)

errors = []
for i in range(1, 11):
  p = func(i)
  errors.append(p)
  
df = sqlContext.createDataFrame(errors, ["trainError","testError", "depth"])
display(df)


De acuerdo a la anterior gráfica el valor con menor complejidad pero que tiene buenos resultados en el conjunto de prueba es profundidad 3.

# 5. Comparación de modelos

In [16]:
display(test)

In [17]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier, NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder


numFolds = 10

labelIndexer = StringIndexer(inputCol="class", outputCol="label").fit(data)

nb = NaiveBayes(labelCol="label", featuresCol="features")
f1_metric = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")  
pipeline = Pipeline(stages=[labelIndexer, nb])
grid = ParamGridBuilder().build()
nb_crossval = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=grid,
    evaluator=f1_metric,
    numFolds=numFolds)

nb_model = nb_crossval.fit(train)
nb_prediction=nb_model.bestModel.transform(test)

print "Naïve Bayes Classifier Metrics"
print "avg. F1-score", nb_model.avgMetrics
print "Best Model Metrics"
print "Accuracy: ",accuracy_metric.evaluate(nb_prediction)
print "Recall:", recall_metric.evaluate(nb_prediction)
print "F1:", f1_metric.evaluate(nb_prediction)

In [18]:
numFolds = 10

rf = RandomForestClassifier(numTrees=5, maxDepth=3, labelCol="label", featuresCol="features")
pipeline = Pipeline(stages=[labelIndexer, rf])
grid = ParamGridBuilder().build()
rf_crossval = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=grid,
    evaluator=f1_metric,
    numFolds=numFolds)

rf_model = rf_crossval.fit(train)
rf_prediction=rf_model.bestModel.transform(test)

print "Random Forest Classifier Metrics"
print "avg. F1-score", rf_model.avgMetrics
print "Best Model Metrics"
print "Accuracy: ",accuracy_metric.evaluate(rf_prediction)
print "Recall:", recall_metric.evaluate(rf_prediction)
print "F1:", f1_metric.evaluate(rf_prediction)


Como se puede apreciar, el modelo de Random Forest fue mejor que el de Naïve Bayes, almenos para los parámetros establecidos para uno y otro, faltaría hacer una exploración de hiperparámetros para ambos modelos ya que existe la posibilidad de que la elección de estos para el modelo bayesiano no sean los mejores para el conjunto de datos usado.

# 6. Cargue el conjunto de datos credit_german

In [21]:
data = sqlContext.read.csv("/FileStore/tables/m1jhk51i1479677154408/credit_german-8665c.csv", header=True, inferSchema=True, sep=';')
display(data)

In [22]:
columns = list(data.dtypes)
features = []
data = data.na.drop()

for col, typ in columns:
  if col == "class":
    continue
  if typ == "string":
    idxCol = col+"Idx"
    feaCol = col+"Feature"
    features.append(feaCol)
    stringIndexer = StringIndexer(inputCol = col, outputCol = idxCol)
    indexer = stringIndexer.fit(data)
    data = indexer.transform(data)
    
    encoder = OneHotEncoder(dropLast = False, inputCol = idxCol, outputCol = feaCol)
    data = encoder.transform(data)
  else:
    assembler = VectorAssembler(inputCols = [col], outputCol = str(col+"Num"))
    data = assembler.transform(data)
    features.append(col)
    
assembler = VectorAssembler(inputCols = features, outputCol = "features")
data = assembler.transform(data)
data = data.select("class", "features")
display(data)

In [23]:
(train, test) = data.randomSplit([0.7, 0.3])

numFolds = 10

labelIndexer = StringIndexer(inputCol="class", outputCol="label").fit(data)

rf = RandomForestClassifier(numTrees=10, maxDepth=5, labelCol="label", featuresCol="features")
f1_metric = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")  
pipeline = Pipeline(stages=[labelIndexer, rf])
grid = ParamGridBuilder().build()
rf_crossval = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=grid,
    evaluator=f1_metric,
    numFolds=numFolds)

rf_model = rf_crossval.fit(train)
rf_prediction=rf_model.bestModel.transform(test)

print "Random Forest Classifier Metrics"
print "avg. F1-score", rf_model.avgMetrics
print "Best Model Metrics"
print "Accuracy: ",accuracy_metric.evaluate(rf_prediction)
print "Recall:", recall_metric.evaluate(rf_prediction)
print "F1:", f1_metric.evaluate(rf_prediction)


In [24]:
display(rf_prediction)

In [25]:

from pyspark.mllib.evaluation import MulticlassMetrics

metrics = MulticlassMetrics(rf_prediction.select('prediction', 'label').rdd)

print "Confusion Matrix"
print pd.DataFrame(metrics.confusionMatrix().toArray(), columns=['good', 'bad'], index=['good', 'bad'])

print "Accuracy", metrics.accuracy
print "Precision", metrics.precision(label=1)
print "F1-Score", metrics.fMeasure(label=1.0)
print "Recall", metrics.recall(label=1)


Como se puede observar, a pesar que a primera vista podria verse que el accuracy es relativamente alto **(0.74)** lo que realmente nos podría interesar es clasificar los clientes de la categoría *"bad"* por lo que analizando métricas como el **Recall** podemos ver que es un mal modelo para esta tarea, lo que se refuerza aún más con el **F1-Score** que también es bajo. Por lo tanto este modelo debe ser descartado, podría hacerse una ajuste de parámetros intentando subir el Recall para la clase 1, pero no es alentador el horizonte al menos como se puede observar para el modelo descrito.